In [18]:
import os
import argparse
import logging
import math
import scipy

import sys
sys.path.append('/nfs/juhu/data/rakhasan/bystander-detection/code-repos/pose-tensorflow-master/')
base_path='/nfs/juhu/data/rakhasan/bystander-detection/code-repos/pose-tensorflow-master/'

from config import load_config,cfg_from_file
from dataset.factory import create as create_dataset
from dataset.pose_dataset import Batch, merge_batch
from nnet.predict import *
from util import visualize
from multiperson.posenms import pose_nms
from predict_2stage import write_posetrack_predictions

import matplotlib.pyplot as plt

os.chdir(base_path)

In [11]:
def get_base_name(path):
    split = path.split(".")
    assert(len(split) == 2)
    base_path = split[0]
    return base_path.replace("/", "_")


In [33]:
cfg = cfg_from_file('pose_cfg.yaml')
dataset = create_dataset(cfg)
dataset.set_shuffle(False)
dataset.set_test_mode(True)
num_images = len(dataset.data)

tf.reset_default_graph() 
sess, inputs, outputs = setup_pose_prediction(cfg)

all_poses = []
all_box_conf = []
all_boxes = []

out_dir = cfg.scoremap_dir
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

(500,)
total_num_persons 0
INFO:tensorflow:Restoring parameters from data/snapshot-250000


In [42]:
k=0
item = dataset.data[k]

boxes = item.boxes
box_conf = item.box_conf
valid_boxes = box_conf >= cfg.person_min_conf
boxes = boxes[valid_boxes]
box_conf = box_conf[valid_boxes]
num_boxes = boxes.shape[0]
num_batches = int(math.ceil(num_boxes / cfg.batch_size))

poses = np.zeros((0, cfg.num_joints, 3), dtype=np.float64)
sm_size = np.ceil(cfg.crop_size / (cfg.stride * 2)).astype(int) * 2
scoremaps = np.zeros((0, sm_size, sm_size, cfg.num_joints), dtype=np.float32)
locref_maps = np.zeros((0, sm_size, sm_size, cfg.num_joints, 2), dtype=np.float32)
scales = np.zeros((0), dtype=np.float32)
top_lefts = np.zeros((0, 2), dtype=np.float32)

In [48]:
for j in range(num_batches):
    start = j * cfg.batch_size
    end = min(start + cfg.batch_size, num_boxes)
    batches = []
    for i in range(start, end):
        item.person_idx = i
        batch = dataset.make_batch(item, cfg.global_scale, mirror=False)
        assert(batch is not None)
        batches.append(batch)
    batch = batches[0] if len(batches) == 1 else merge_batch(batches)

    inputs_to_net = batch[Batch.inputs]

In [59]:
outputs_np = sess.run(outputs, feed_dict={inputs: inputs_to_net})

In [63]:
type(outputs_np)
outputs_np.keys()

dict_keys(['part_prob', 'locref'])

In [64]:
def extract_cnn_output2(outputs_np, cfg, pairwise_stats = None):
    scmap = outputs_np['part_prob']
    scmap = np.squeeze(scmap)
    locref = None
    pairwise_diff = None
    if cfg.location_refinement:
        locref = np.squeeze(outputs_np['locref'])
        shape = locref.shape
        locref = np.reshape(locref, (shape[0], shape[1], -1, 2))
        locref *= cfg.locref_stdev
    if cfg.pairwise_predict:
        pairwise_diff = np.squeeze(outputs_np['pairwise_pred'])
        shape = pairwise_diff.shape
        pairwise_diff = np.reshape(pairwise_diff, (shape[0], shape[1], -1, 2))
        num_joints = cfg.num_joints
        for pair in pairwise_stats:
            pair_id = (num_joints - 1) * pair[0] + pair[1] - int(pair[0] < pair[1])
            pairwise_diff[:, :, pair_id, 0] *= pairwise_stats[pair]["std"][0]
            pairwise_diff[:, :, pair_id, 0] += pairwise_stats[pair]["mean"][0]
            pairwise_diff[:, :, pair_id, 1] *= pairwise_stats[pair]["std"][1]
            pairwise_diff[:, :, pair_id, 1] += pairwise_stats[pair]["mean"][1]
    return scmap, locref, pairwise_diff


In [71]:
idx=0
output_np = {
            "part_prob" : outputs_np["part_prob"][idx,:,:,:],
            "locref": outputs_np["locref"][idx, :, :, :]
            }
output_np['part_prob'].shape,output_np['locref'].shape

((42, 42, 14), (42, 42, 28))

In [72]:
scmap, locref, _ = extract_cnn_output2(output_np, cfg)
pose = argmax_pose_predict(scmap, locref, cfg.stride)

In [79]:
pose.shape,scmap.shape

((14, 3), (42, 42, 14))

In [4]:

def test_net(visualise, cache_scoremaps, development):
    logging.basicConfig(level=logging.DEBUG)

    cfg = load_config()
    dataset = create_dataset(cfg)
    dataset.set_shuffle(False)
    dataset.set_test_mode(True)

    num_images = len(dataset.data)

    sess, inputs, outputs = setup_pose_prediction(cfg)

    all_poses = []
    all_box_conf = []
    all_boxes = []

    out_dir = cfg.scoremap_dir
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    for k in range(num_images):
        print('processing image {}/{}'.format(k, num_images-1))

        item = dataset.data[k]

        boxes = item.boxes
        box_conf = item.box_conf
        valid_boxes = box_conf >= cfg.person_min_conf
        boxes = boxes[valid_boxes]
        box_conf = box_conf[valid_boxes]
        num_boxes = boxes.shape[0]
        num_batches = int(math.ceil(num_boxes / cfg.batch_size))

        poses = np.zeros((0, cfg.num_joints, 3), dtype=np.float64)

        sm_size = np.ceil(cfg.crop_size / (cfg.stride * 2)).astype(int) * 2
        scoremaps = np.zeros((0, sm_size, sm_size, cfg.num_joints), dtype=np.float32)
        locref_maps = np.zeros((0, sm_size, sm_size, cfg.num_joints, 2), dtype=np.float32)
        scales = np.zeros((0), dtype=np.float32)
        top_lefts = np.zeros((0, 2), dtype=np.float32)

        for j in range(num_batches):
            start = j * cfg.batch_size
            end = min(start + cfg.batch_size, num_boxes)
            batches = []
            for i in range(start, end):
                item.person_idx = i
                batch = dataset.make_batch(item, cfg.global_scale, mirror=False)
                assert(batch is not None)
                batches.append(batch)
            batch = batches[0] if len(batches) == 1 else merge_batch(batches)

            inputs_to_net = batch[Batch.inputs]
            """
            print(inputs_to_net.shape)
            num_in_batch = inputs_to_net.shape
            if num_in_batch < cfg.batch_size:
                inputs_to_net[cfg.batch_size-1,0,0,0] = 0
            """
            outputs_np = sess.run(outputs, feed_dict={inputs: inputs_to_net})

            for idx, i in enumerate(range(start, end)):
                item.person_idx = i

                output_np = {
                    "part_prob" : outputs_np["part_prob"][idx,:,:,:],
                    "locref": outputs_np["locref"][idx, :, :, :]
                }
                #output_np = {k: v[idx,:,:,:] for (k, v) in outputs_np}

                scmap, locref, _ = extract_cnn_output(output_np, cfg)
                pose = argmax_pose_predict(scmap, locref, cfg.stride)

                #img = np.squeeze(batch[Batch.inputs][idx, :, :, :]).astype('uint8')
                #visualize.show_heatmaps(cfg, img, scmap, pose)
                #visualize.waitforbuttonpress()

                scale = batch[Batch.scale][idx]
                top_left = batch[Batch.crop_topleft][idx]
                pose[:,[0,1]] /= scale
                pose[:,[0,1]] += top_left

                pose = np.expand_dims(pose, axis=0)
                poses = np.concatenate((poses, pose), axis=0)

                scoremaps = np.concatenate((scoremaps, np.expand_dims(scmap, axis=0)), axis=0)
                locref_maps = np.concatenate((locref_maps, np.expand_dims(locref, axis=0)), axis=0)
                scales = np.concatenate((scales, np.array([scale])), axis=0)
                top_lefts = np.concatenate((top_lefts, np.expand_dims(top_left, axis=0)), axis=0)

        #base_name = os.path.splitext(os.path.basename(item.im_file))[0]
        base_name = get_base_name(item.im_file)
        if cache_scoremaps:
            cache_name = "{}.mat".format(base_name)

            out_fn = os.path.join(out_dir, cache_name)
            dict = {'scoremaps': scoremaps.astype('float32'),
                    'locreg_pred': locref_maps.astype('float32'),
                    'scale': scales, 'top_left': top_lefts}
            scipy.io.savemat(out_fn, mdict=dict)

        if cfg.pose_nms:
            poses, box_conf, boxes = pose_nms(poses, box_conf, boxes, cfg)

        poses_out_fn = os.path.join(out_dir, "{}_poses.mat".format(base_name))
        scipy.io.savemat(poses_out_fn, mdict={"poses": poses})

        all_poses.append(poses)
        all_box_conf.append(box_conf)
        all_boxes.append(boxes)

    np.savez("predictions.npz", all_poses, all_box_conf, all_boxes)
    if "dataset_sequences" in cfg and cfg.dataset_sequences:
        write_posetrack_predictions(cfg, all_poses, all_box_conf, all_boxes)

    sess.close()

In [ ]:
+